In [400]:
import itertools

In [401]:
def All(xs,p):
    return all(itertools.imap(p,xs))

def Some(xs,p):
    return any(itertools.imap(p,xs))

In [440]:
def log2(b):
    bitCnt = 0
    while b:
        bitCnt += 1
        b >>= 1
    return bitCnt

def log16(b):
    hexdCnt = 0
    if b:
        while b:
            hexdCnt += 1
            b >>= 4
        return hexdCnt
    else:
        return 1

def bitCnt(n):
    b = 0
    while n:
        if n&1:
            b += 1
        n >>= 1
    return b

def bits(n):
    b = 0
    while n:
        if n&1:
            yield b
        b += 1
        n >>=1
        
def toFr(n):
    return frozenset(bits(n))

def fromFr(f):
    return sum(1<<b for b in f)

def elmtCmp(x,y):
    diff = bitCnt(x) - bitCnt(y)
    if diff:
        return diff
    else:
        return x - y

hexDict = {
    '0':'0000', '1':'0001', '2':'0010', '3':'0011', '4':'0100', '5':'0101',
    '6':'0110', '7':'0111', '8':'1000', '9':'1001', 'a':'1010', 'b':'1011',
    'c':'1100', 'd':'1101', 'e':'1110', 'f':'1111', 'L':''}

def nat2bin(n, width=8):
    """
    A foolishly simple look-up method of getting binary string from an integer
    This happens to be faster than more sophisticated other ways!!! Stolen from I do not remember where.
    """
    if n == None:
        return ""
    # =========================================================
    # create hex of int, remove '0x'. now for each hex char,
    # look up binary string, append in list and join at the end.
    # =========================================================
    hstr = hex(n)[2:]
    extra = max(width - 4*len(hstr), 0)
    return (extra*'0')+ "".join(hexDict[hch] for hch in hstr)

def nat2hex(n,width=2):
    s = hex(n)[2:].upper()
    extras = max(width-len(s),0)
    return extras*'0' + s

    
class CarrierSet:
    def __init__(self,xs, d=None):
        self._elmts = frozenset(xs)

        if d:
            self._dim = d
        elif len(self._elmts) == 0:
            self.dim = 1
        else:
            self._dim = max(log2(x) for x in self._elmts)
        self._list = sorted(self._elmts,elmtCmp)   
        
    def __len__(self):
        return len(self._elmts)
    
    def dim(self):
        return self._dim
    
    def __call__(self,x):
        return x in self._elmts
    
    def __iter__(self):
        return iter(self._list)
    
    def getSubset(self,p):
        return frozenset(self._list[b] for b in bits(p))
    
    def col(self,k):
        res = 0
        b = 1
        m = 1<<k
        for e in self._list:
            if e&m:
                res |= b
            b <<= 1
        return res
    
    def _repr_latex_(self):
        if len(self) == 0:
            return r"$\emptyset$"
        s = r"$\{\textsf{%s}"%nat2bin(self._list[0], self._dim)
        for x in self._list[1:]:
            s += r", \textsf{%s}"%nat2bin(x,self._dim)
        s += r"\}$"
        return s

def powerSet(cs):
    s = list(cs)
    return itertools.imap(frozenset, 
                           itertools.chain.from_iterable(itertools.combinations(s, r) for r in range(len(s)+1)))


In [442]:
CarrierSet([2,3,4])

<__main__.CarrierSet instance at 0x10665f5f0>

In [404]:
def isGOA(xs):
    if 0 not in xs:
        return False
    
    return All(itertools.combinations(xs,2),lambda (x,y):x&y or (x|y in xs))

def isGDP(xs):
    if 0 not in xs:
        return False
    
    return All(itertools.permutations(xs,2),lambda (x,y):(x&y != y) or (x-y in xs))


In [405]:
def ortho(a,b):
    return not (a&b)

def oplus(a,b):
    if ortho(a,b):
        return a|b
    else:
        return None
    
def leq(a,b):
    return a==a&b

def geq(a,b):
    return a&b == b

def gt(a,b):
    return a!=b and a&b == b

def ominus(a,b):
    if geq(a,b):
        return a&~b
    else:
        return None

# make a 'copy' of the frozen set x with each element b replaced by the set {b*n + i for i in w}
def spread(x,w,n):
    return frozenset(b*n + i for b in x for i in w)
            

In [407]:
tupleInReln(J3,[0x0,0x0,0x0],4)

True

In [408]:
class GOA(CarrierSet):
    def __init__(self, g):
        CarrierSet.__init__(self,g)
        if not isGOA(self):
            raise ArithmeticError()
        self._relns = []

    def __mul__(self,B):
        d = self.dim()
        return GOA([(b<<d) | a for a in self for b in B])
    
    def __add__(self,B):
        d = 1<<B.dim()
        return GOA(map(lambda x:spread(x,2,1), self) + map(lambda x:spreadBits(x,2,3), B))
       
    def _repr_latex_(self):
        w = log16(self.dim())

        s = r"\[\begin{array}{r|"+(len(self._elmts)*"r"+"}")
        s += r" \oplus "
        
        ls = sorted(self._list)
        for x in ls:
            s += r" & \textsf{%s}"%nat2hex(x, w)
        s += r"\\ \hline "
        for x in ls:
            s += " " + r"\textsf{%s}"%nat2hex(x,w)
            for y in ls:
                s += " & "
                xoy = oplus(x,y)
                if xoy != None:
                    s += r"\textsf{%s}"%nat2hex(xoy,w)
            s += r" \\"
        s += r"\end{array}\]"
        return s

    def isPrime(self,p):

        f = self.getSubset(p)
        if 0 in f: return False
        for (x,y) in itertools.combinations(self._elmts,2):
            if ortho(x,y):
                fx = x in f
                fy = y in f
                if (fx and fy) or ((fx or fy) != (oplus(x,y) in f)):
                    return False
        
        for R in GDP._relns:
            for t in allTuples(R,self._elmts, self._dim):
                if not preservesReln(R,t,f):
                    return False

        return True
    
    def spectrum(self):
        return GDP(p for p in range(1<<(len(self)-1)) if self.isPrime(p<<1))
 


In [427]:
class GDP(CarrierSet):
    def __init__(self, g):
        CarrierSet.__init__(self,g)
        if not isGDP(self._elmts):
            raise ArithmeticError()
        GDP._relns = [J3]
        
    def __mul__(self,B):
        d = self.dim()
        return GDP([(b<<d) | a for a in self for b in B])
    
    def __add__(self,B):
        d = 1<<B.dim()
        return GDP(map(lambda x:spread(x,2,1), self) + map(lambda x:spreadBits(x,2,3), B))
       
    def _repr_latex_(self):
        w = log16(self.dim())

        s = r"\[\begin{array}{r|"+(len(self._elmts)*"r"+"}")
        s += r" \ominus "
        
        ls = sorted(self._list)
        for x in ls:
            s += r" & \textsf{%s}"%nat2hex(x, w)
        s += r"\\ \hline "
        for x in ls:
            s += " " + r"\textsf{%s}"%nat2hex(x,w)
            for y in ls:
                s += " & "
                xoy = ominus(x,y)
                if xoy != None:
                    s += r"\textsf{%s}"%nat2hex(xoy,w)
            s += r" \\"
        s += r"\end{array}\]"
        return s
                
    def isPrime(self,p):
        f = self.getSubset(p)
        if 0 in f: return False
        for (x,y) in itertools.permutations(self._elmts,2):
            if gt(x,y):
                fx = x in f
                fy = y in f
                if ((not fx) and fy) or ((fx and (not fy) != (ominus(x,y) in f))):
                    return False

        for R in GDP._relns:
            for t in allTuples(R,self._elmts,self._dim):
                if not preservesReln(R,t,f):
                    return False
        print p
        return True
    
    def spectrum(self):
        return GOA(p for p in range(1<<(len(self)-1)) if self.isPrime(p<<1))
 


In [428]:
OPAR_4 = GOA([0b0000,0b1100,0b0011,0b1010,0b0101,0b1001,0b0110, 0b1111])
DPAR_4 = GDP([0b0000,0b1100,0b0011,0b1010,0b0101,0b1001,0b0110, 0b1111])

In [436]:
DPAR_4.spectrum()

ArithmeticError: 

In [430]:
J3

<__main__.CarrierSet instance at 0x106615320>

In [431]:
len(list(allTuples(J3,DS_4,4)))

6561

In [432]:
P_1 = GOA([0,1])
SQ_4 = P_1*P_1*P_1*P_1
SQ_4

<__main__.GOA instance at 0x1053f5050>

In [433]:
DPAR_4

<__main__.GDP instance at 0x1053e3320>

In [434]:
OPAR_4

<__main__.GOA instance at 0x1053e3e60>

In [435]:
OPAR_4.spectrum()

ArithmeticError: 